In [1]:
import pysb.bng
import kholodenko_basic as kb
import kholodenko_m1 as m
import sympy as sp
from sympy.printing import latex
pysb.bng.__file__

'/home/priya/pysb/pysb/bng.pyc'

In [2]:
pysb.bng.generate_equations(kb.model)
pysb.bng.generate_equations(m.model)

In [3]:
"""for i, j in zip(kb.model.species, ke.model.species):
 
    assert i.is_equivalent_to(j)

species_map = dict(zip(range(len(kb.model.species)), kb.model.species))

ke_subs, kb_subs = [zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions]) for m in ke, kb]
print ke.model.parameters[2]
ke_dict, kb_dict = [dict([((x['reactants'], x['products']), x['rate']) for x in y])
                                    for y in [ke.model.reactions, kb.model.reactions]]

"""
"""
TODO:
1. Substitute the species __s0 with some identifier
2. Find a list of all of the UNIQUE bidirectional reactions"""

def print_rct_rates(m):
    for (react, prod), (fwd, rev, kd) in gen_rct_rates(m):
        print react, "<-->", prod
        print "K_f:", fwd
        print "K_r:", rev
        print "K_d:", kd
        
def print_to_latex(m, outfile):
    for (react, prod), (fwd, rev, kd) in gen_rct_rates(m):
        outfile.write(str(react).replace("%", ":") + "<-->" + str(prod).replace("%", ":") + " \\ \n")
        outfile.write(latex(fwd) + "\\ \n")
        outfile.write(latex(rev) + "\\ \n")
        outfile.write(latex(kd) + "\\ \n")
        
def gen_rct_rates(m):
    ## from a model, create a generator of all reaction rates binned by forward, reverse, kd
    rct_dict = dict([((x['reactants'], x['products']), x['rate']) for x in m.model.reactions])
    rct_subs = zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions])
    species_map = dict(zip(range(len(m.model.species)), m.model.species))
    species = lambda tup: [species_map[x] for x in tup]
    fwd_keys = sorted(filter(lambda k: len(k[0]) == 2, rct_dict.keys()))

    for k in fwd_keys:
        yield ((species(k[0]), species(k[1])), rct_rates(k, rct_dict, rct_subs))

def rct_rates(fwd_key, rct_dict, rct_subs):
    ## Takes keys of FORWARD reactions
    fwd_rate = extra_simple(rct_dict[fwd_key].subs(rct_subs).simplify())
    rev_rate = extra_simple(rct_dict[(fwd_key[1], fwd_key[0])].subs(rct_subs).simplify())
    k_d = extra_simple(rev_rate/fwd_rate)
    return (fwd_rate, rev_rate, k_d)

def extra_simple(mul):
    return sp.expand_power_base(sp.powdenest(sp.logcombine(sp.expand_log(mul.simplify(), force=True), force=True), force=True), force=True)

print_rct_rates(m)
print_to_latex(m, open('ke_model.tex', 'w'))
"""for key in sorted(ke_dict.keys()):
    #e_sub = sp.sympify(str(ke_dict[key].subs(ke_subs).simplify()))
    #b_sub = sp.sympify(str(kb_dict[key].subs(kb_subs).simplify()))
    e_sub = extra_simple(ke_dict[key].subs(ke_subs).simplify())
    b_sub = extra_simple(kb_dict[key].subs(kb_subs).simplify())

    assert e_sub.equals(b_sub)

    print ""
    print "rct:", key[0],"-->", key[1]
    print "rct:", species(key[0]),"-->", species(key[1])
    print "e", e_sub
    #print "eunsimp", ke_dict[key].subs(ke_subs)
    print "b", b_sub
    
    #print(latex(e_sub))
    #print(latex(b_sub))
    print ""
    #assert e_sub.equals(b_sub)

    
    #print e.subs(ke_subs), b.subs(kb_subs)
#for e, b in rct:
    #e_sub = sp.sympify(str(e[2].subs(ke_subs).simplify()))
    #b_sub = sp.sympify(str(b[2].subs(kb_subs).simplify()))"""

[B(d=None, i=None), B(d=None, i=None)] <--> [B(d=1, i=None) % B(d=1, i=None)]
K_f: 0.5*__s0**2*kon_BB
K_r: __s2*koff_BB
K_d: 2.0*__s2*kon_BB**(-1)*koff_BB/__s0**2
[B(d=None, i=None), I(b=None)] <--> [B(d=None, i=1) % I(b=1)]
K_f: __s0*__s1*kon_BI
K_r: __s3*koff_BI
K_d: __s3*kon_BI**(-1)*koff_BI/(__s0*__s1)
[B(d=None, i=None), B(d=None, i=1) % I(b=1)] <--> [B(d=1, i=2) % B(d=1, i=None) % I(b=2)]
K_f: __s0*__s3*f**(-phi)*kon_BB
K_r: __s4*f**(1 - phi)*koff_BB
K_d: __s4*kon_BB**(-1)*f*koff_BB/(__s0*__s3)
[I(b=None), B(d=1, i=None) % B(d=1, i=None)] <--> [B(d=1, i=2) % B(d=1, i=None) % I(b=2)]
K_f: 2.0*__s1*__s2*f**(-phi)*kon_BI
K_r: __s4*f**(1 - phi)*koff_BI
K_d: 0.5*__s4*kon_BI**(-1)*f*koff_BI/(__s1*__s2)
[I(b=None), B(d=1, i=2) % B(d=1, i=None) % I(b=2)] <--> [B(d=1, i=2) % B(d=1, i=3) % I(b=2) % I(b=3)]
K_f: __s1*__s4*(f**(-1)*g**(-1))**phi*kon_BI
K_r: 2.0*__s5*(f*g)**(1 - phi)*koff_BI
K_d: 2.0*__s5*kon_BI**(-1)*f*g*koff_BI/(__s1*__s4)
[B(d=None, i=1) % I(b=1), B(d=None, i=1) % I(b=1)] 

'for key in sorted(ke_dict.keys()):\n    #e_sub = sp.sympify(str(ke_dict[key].subs(ke_subs).simplify()))\n    #b_sub = sp.sympify(str(kb_dict[key].subs(kb_subs).simplify()))\n    e_sub = extra_simple(ke_dict[key].subs(ke_subs).simplify())\n    b_sub = extra_simple(kb_dict[key].subs(kb_subs).simplify())\n\n    assert e_sub.equals(b_sub)\n\n    print ""\n    print "rct:", key[0],"-->", key[1]\n    print "rct:", species(key[0]),"-->", species(key[1])\n    print "e", e_sub\n    #print "eunsimp", ke_dict[key].subs(ke_subs)\n    print "b", b_sub\n    \n    #print(latex(e_sub))\n    #print(latex(b_sub))\n    print ""\n    #assert e_sub.equals(b_sub)\n\n    \n    #print e.subs(ke_subs), b.subs(kb_subs)\n#for e, b in rct:\n    #e_sub = sp.sympify(str(e[2].subs(ke_subs).simplify()))\n    #b_sub = sp.sympify(str(b[2].subs(kb_subs).simplify()))'

### dir(m)

In [4]:
m.model.reactions_bidirectional


[{'products': (2,),
  'rate': 0.5*__s0**2*_BB_dimerization_local1 - __s2*__reverse_BB_dimerization_local1,
  'reactants': (0, 0),
  'reversible': True,
  'rule': ('BB_dimerization',)},
 {'products': (3,),
  'rate': __s0*__s1*_BI_binding_local1 - __s3*__reverse_BI_binding_local1,
  'reactants': (0, 1),
  'reversible': True,
  'rule': ('BI_binding',)},
 {'products': (4,),
  'rate': __s0*__s3*_BB_dimerization_local2 - __s4*__reverse_BB_dimerization_local2,
  'reactants': (0, 3),
  'reversible': True,
  'rule': ('BB_dimerization',)},
 {'products': (5,),
  'rate': 0.5*__s3**2*_BB_dimerization_local3 - __s5*__reverse_BB_dimerization_local3,
  'reactants': (3, 3),
  'reversible': True,
  'rule': ('BB_dimerization',)},
 {'products': (4,),
  'rate': 2.0*__s1*__s2*_BI_binding_local2 - __s4*__reverse_BI_binding_local2,
  'reactants': (1, 2),
  'reversible': True,
  'rule': ('BI_binding',)},
 {'products': (5,),
  'rate': __s1*__s4*_BI_binding_local3 - 2.0*__s5*__reverse_BI_binding_local3,
  'react

In [5]:
sp.parsing.sympy_parser.eval_expr('0.5*__s0**2*exp(RT*log(kon_BB)*RT**(-1))', {}, {})

NameError: name '__s0' is not defined